![Top <](./images/watsonxdata.png "watsonxdata")

# Pandas Code example
This code will connect to watsonx.data using a Pandas dataframe and display some data from an existing table that was created in Presto. Note that the certificate is available in this Jupyter notebook environment in the /certs directory.

Import the libraries we need to connect.

In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

The following cell contains all of the credentials we need to connect to the watsonx.data database. Note that this code is running inside the watsonx.data server. This means that the port number used here is for the internal connection, not the external one. If you want to connect from a Jupyter notebook "outside" the watsonx.data server, you would use the following settings:

* Hostname - The name of the techzone server that the watsonx.data system was provisioned on.
* Port - The port number on the server (which will be different than 8443, but it will map to the 8443 port when it connects to the server).
* Certfile - This must be the absolute location of the certificate file

You need to extract the lh-ssl-ts.crt file directly from the watsonx.data image by using the following syntax:
```
scp -P <port> watsonx@<server>:/cert/lh-ssl-ts.crt <file_location>
```
Replace the following values in the string above:
* `<port>` - The port number that was provided in your reservation for SSH commands
* `<server>` - The name of the server that the watsonx.data image is running on
* `<file_location>` - The location on your local file system that you want to store this file.

In [ ]:
userid    = "ibmlhadmin"
password  = "password"
hostname  = "watsonxdata"
port      = "8443"
catalog   = "tpch"
schema    = "tiny"
certfile  = "/certs/lh-ssl-ts.crt"
connect_args={
        'protocol': 'https', 
        'requests_kwargs': {'verify': f'{certfile}'}
        }

Create the engine connect to Presto.

In [ ]:
engine = create_engine(
   f"presto://{userid}:{password}@{hostname}:{port}/{catalog}/{schema}",
   connect_args=connect_args
   )

The next statement will execute a query and retrieve the entire result set.

In [ ]:
mypresto = pd.read_sql_query('SELECT * from tpch.tiny.customer',engine)

The variable `mypresto` is a pandas dataframe that you can manipulate or display. The following code will display the dataframe in a scrollable window.

In [ ]:
from ipydatagrid import DataGrid
DataGrid(mypresto)

You can use other program like `matplotlib` to create plots on data found in the result set.

In [ ]:
import matplotlib
sumbynation = pd.read_sql_query('SELECT "c_nationkey", sum("c_acctbal") as total from tpch.tiny.customer group by "c_nationkey" order by 2',engine)
sumbynation.plot(kind="bar",x="c_nationkey",y="total")

#### Credits: IBM 2025, George Baklarz [baklarz@ca.ibm.com]